In [ ]:
import requests
import time
from tqdm._tqdm_notebook import tqdm_notebook
import pandas as pd
tqdm_notebook.pandas()
from IPython.display import HTML
pd.set_option('display.max_columns', 500)

# Room.nl reader & helper

<img src="./cookie.png" width="40%" style="float: right; margin-left: 20px" />

Deze tool helpt je bij het snel vinden van je beste kamer in plaats van de angular app op Room.nl te gebruiken. Vul gewoon je leeftijd en inkomen in (voor berekening huurtoeslag) en geef toegang tot Room.nl door je php-sessiecode in te vullen vanuit cookies. 

Om te beginnen run je alle cellen. Bij de eerste cel wordt om drie waardes gevraagd. Vul bij de php-sessie-id de cookie in die in je browser wordt gezet nadat je inlogt (deze is niet lang geldig, zeg 30 minuten), dus doe dit elke keer opnieuw. 

Zie in de afbeelding hiernaast hoe je deze code vindt na het inloggen op Rooms. 

In [ ]:
leeftijd = int(input('Geef hier je leeftijd in een rond getal: '))
income = int(input('Geef hier je geschat inkomen in een rond getal: '))
phpsessid = input('Plak hier je PHP-sessiecode: ')

**De code hieronder kan je voor nu gewoon laten runnen en overslaan. Onderin dit programma staan conclusies.**

# Get raw data
Haal ruwe data op van Room.nl met jouw php-sessiecode.

### Get all rooms available

In [ ]:
cookies = {
    'staticfilecache': 'typo_user_logged_in',
    'PHPSESSID': phpsessid,
}

headers = {
    'Origin': 'https://www.room.nl',
    'Referer': 'https://www.room.nl/aanbod/studentenwoningen/',
    'X-Requested-With': 'XMLHttpRequest',
}
response = requests.post('https://www.room.nl/portal/object/frontend/getallobjects/format/json', headers=headers, cookies=cookies)

# Make dataframe.
result = pd.DataFrame(response.json()['result']).set_index('id')

# Recode to mostly non-dictionary flat items.
result.regio = [d.get('name') for d in result.regio]
result.municipality = [d.get('name') for d in result.municipality]
result.city = [d.get('name') for d in result.city]
result.quarter = [d.get('name') for d in result.quarter]
result.floor = [d.get('localisedName') for d in result.floor]
result.huurtoeslagVoorwaarde = [d.get('localizedIconText') for d in result.huurtoeslagVoorwaarde]
result['amountOfRooms'] = [d.get('amountOfRooms') for d in result.sleepingRoom]
result['isHospiteren'] = [d.get('isHospiteren') for d in result.model]
result.woningsoort = [d.get('localisedName') for d in result.woningsoort]

# Drop columns that do not return any values (or always the same).
result = result.drop(columns=[
    'dwellingType', 'neighborhood', 'availableFrom', 'sellingPrice', 
    'energyLabel', 'sleepingRoom', 'rentBuy', 'closingDate', 'actionLabel', 
    'actionLabelFrom', 'actionLabelUntil', 'actionLabelIfActive', 
    'relatieHuurInkomenData', 'relatieHuurInkomenGroepen', 'koopvoorwaarden', 'vatInclusive',
    'verzameladvertentieID', 'isExternModelType', 'woningsoort', 'reactieUrl',
    'isInGepubliceerdeVerzameladvertentie', 'isGepubliceerd', 'doelgroepen',
    'isExtraAanbod', 'newlyBuild', 'model', 'pictures'
])
aanbiedingen = result

### Get all available offers for this user

In [ ]:
# Get room application information (user dependent)
cookies = {
    'staticfilecache': 'typo_user_logged_in',
    'PHPSESSID': phpsessid,
}

headers = {
    'Origin': 'https://www.room.nl',
    'Referer': 'https://www.room.nl/aanbod/studentenwoningen/',
    'X-Requested-With': 'XMLHttpRequest',
}

data = [
  ('configurationKeys[]', 'aantalReacties'),
  ('configurationKeys[]', 'passend'),
]

response = requests.post('https://www.room.nl/portal/object/frontend/getdynamicdata/format/json', headers=headers, cookies=cookies, data=data)
result = [{**l, **l['reactionData']} for l in response.json()['result']]
application_poss_df = pd.DataFrame(result).set_index('id').drop(columns=['reactionData', 'loggedin', 'aangepasteTotaleHuurprijs', 'aangepasteNettoHuurprijs'])


### Combine both dataframes into a single one

In [ ]:
# Combine two datasets into extended single dataset.
woning_df = aanbiedingen.join(application_poss_df, on='id')
print(woning_df.shape)
woning_df.head(2)

# Voeg huurtoeslag toe
Snelle naieve berekening van je huurtoeslag. Gaat er van uit dat je als student alleen op kamers gaat wonen zonder speciale maatregelen.

In [ ]:
# For 2020.
# Retrieved from https://download.belastingdienst.nl/toeslagen/docs/berekening_huurtoeslag_tg0831z01fd.pdf
# This is making a lot of default assumptions (no handicap, solo living (EP), etc.)
def huurtoeslag(rent, service_costs, age_under_23, income):
    
    max_under_23 = 432.51
    max_over_23 = 737.14
    
    service_costs = min(48, service_costs)
    rekenhuur = rent + service_costs
    
    if age_under_23 and rekenhuur > max_under_23:
        return 0
    
    if not age_under_23 and rekenhuur > max_over_23:
        return 0
    
    factor_a = 0.000000637464
    factor_b = 0.002341779776
    
    minimum_income = 16.650
    
    # Calculate mandatory self-payment-part: 
    if income <= minimum_income:
        basishuur = 232.65
    else:
        basishuur = (factor_a * income)**2 + (factor_b * income) + 16.94
    
    # Boven deze grens gaat er van je huur af (extra duur, extra kwaliteit)
    kwaliteitskortingsgrens = 432.51
    aftoppingsgrens = 619.01
    
    # Calculation:
    part_a = min(kwaliteitskortingsgrens, rekenhuur) - basishuur
    part_b, part_c = 0, 0
    
    if rekenhuur > kwaliteitskortingsgrens:
        part_b = (min(rekenhuur, aftoppingsgrens) - max(basishuur, kwaliteitskortingsgrens)) * 0.65
    
    if rekenhuur > aftoppingsgrens:
        part_c = (rekenhuur - max(basishuur, aftoppingsgrens)) * 0.4
    
    # Wat een gezeik maar het lijkt oke te werken
    return round(max((part_a + part_b + part_c), 0))
    
# Run some tests to see if these are correct.
# Validate here: https://www.belastingdienst.nl/rekenhulpen/toeslagen/
assert huurtoeslag(rent=395, service_costs=35, age_under_23=True, income=0) == 197
assert huurtoeslag(494, service_costs=110, age_under_23=False, income=0) == 271
assert huurtoeslag(580, service_costs=120, age_under_23=False, income=0) == 325

In [ ]:
def apply_huurtoeslag(row):
    row['huurtoeslag'] = huurtoeslag(
        rent=row.netRent, 
        service_costs=min(48, (row.totalRent - row.netRent)),
        age_under_23=(leeftijd < 23),
        income=income
    )
    
    row['huurNaToeslag'] = row.totalRent - row['huurtoeslag']
    
    return row
        

In [ ]:
def linkify(url):
    return '<a target="_blank" rel="noopener noreferrer" href="https://www.room.nl/aanbod/studentenwoningen/details/{}/">Open</a>'.format(url)


# Get some nice conclusions from the data!
Hier kunnen we wat spelen met data.

In [ ]:
# Select all places that are available to react to in new frame: filtered.
woning_df_f = woning_df[woning_df.kanReageren]

# Add link.
woning_df_f = woning_df_f.drop(columns='link', errors='ignore')
woning_df_f.insert(0, 'link', woning_df_f.urlKey.apply(linkify))

# Apply huurtoeslag
woning_df_f = woning_df_f.apply(apply_huurtoeslag, axis=1)

# Filter on some unwanted in info-field
woning_df_f = woning_df_f[~woning_df_f.infoveldKort.str.contains('2020')]

# Remove if allow for ending same year, it looks for day-month like 03-11 or 23-1
woning_df_f = woning_df_f[~woning_df_f.infoveldKort.str.contains(' \d+\-\d+!*$')]


### Get possible applications and possible rankings

In [ ]:
def get_application_info(id):
    
    # Make it look a bit less artificial...
    time.sleep(0.1)
    
    cookies = {
        'staticfilecache': 'typo_user_logged_in',
        'PHPSESSID': phpsessid,
    }
    
    headers = {
        'Host': 'www.room.nl',
        'Origin': 'https://www.room.nl',
        'X-Requested-With': 'XMLHttpRequest',
    }

    data = {
      'id': str(id)
    }

    return requests.post(
        'https://www.room.nl/portal/object/frontend/getobject/format/json', 
        headers=headers, 
        cookies=cookies, 
        data=data).json()['result']['reactionData']


In [ ]:
def apply_app_info(row):
    info = get_application_info(row.name)
    row['voorlopigePositie'] = info['voorlopigePositie']
    row['mogelijkePositie'] = info['mogelijkePositie']
    if row['mogelijkePositie'] == None:
        row['mogelijkePositie'] = info['voorlopigePositie']
    return row

woning_df_f = woning_df_f.progress_apply(apply_app_info, axis=1)

## Kamers met de laagste huur na huurtoeslag
En hier kunnen we dan de resultaten zien: de kamer die we het beste kunnen nemen! Eerst laten we zien welke opties we hebben, vervolgens laten we een gesorteerde versie zien.

In [ ]:
woning_df_f.head(1)

In [ ]:
columns = [
    'link', 'infoveldKort', 'street', 'houseNumber', 'aantalMedebewoners', 
    'quarter', 'huurNaToeslag', 'mogelijkePositie', 'voorlopigePositie'
]

HTML(
    woning_df_f[columns].sort_values(by=['mogelijkePositie', 'huurNaToeslag']).to_html(escape=False)
)